In [1]:
# Troubleshooting Retrieval Issues
# Checking for stale chunks in DuckDB and ChromaDB

import os
import sys
from pathlib import Path
import pandas as pd

# Add project root to path 
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(project_root))

print(f"Project root: {project_root}")
print(f"Working from: {Path.cwd()}")

# Expected chunk count from latest successful run
EXPECTED_CHUNKS = 4165  # From chunks_for_embedding_summary.csv (4167 lines - 1 header)


Project root: /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025
Working from: /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/notebooks


In [2]:
# Check DuckDB chunk count
from api.utils.duckdb_utils import get_duckdb_helper

print("=== CHECKING DUCKDB ===")
db_helper = get_duckdb_helper(str(project_root / "artifacts" / "mdc_challenge.db"))

# Get database statistics
stats = db_helper.get_database_stats()
print(f"📊 Database Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

actual_chunks_db = stats['total_chunks']
print(f"\n🔍 Expected chunks: {EXPECTED_CHUNKS}")
print(f"🔍 Actual chunks in DuckDB: {actual_chunks_db}")

if actual_chunks_db == EXPECTED_CHUNKS:
    print("✅ DuckDB chunk count matches expected!")
elif actual_chunks_db > EXPECTED_CHUNKS:
    print(f"⚠️  DuckDB has {actual_chunks_db - EXPECTED_CHUNKS} extra chunks (stale data)")
else:
    print(f"❌ DuckDB is missing {EXPECTED_CHUNKS - actual_chunks_db} chunks")


2025-08-01 10:36:45,134 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/duckdb_utils.log
2025-08-01 10:36:45,134 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/duckdb_utils.log
2025-08-01 10:36:45,198 - src.helpers - INFO - Database schema initialized successfully
2025-08-01 10:36:45,198 - src.helpers - INFO - Database schema initialized successfully


=== CHECKING DUCKDB ===
📊 Database Statistics:
  total_documents: 524
  total_citations: 487
  total_chunks: 4205
  documents_with_citations: 0

🔍 Expected chunks: 4165
🔍 Actual chunks in DuckDB: 4205
⚠️  DuckDB has 40 extra chunks (stale data)


In [3]:
# Check ChromaDB chunk count
import chromadb
import yaml

print("\n=== CHECKING CHROMADB ===")

# Load config to get ChromaDB path
config_path = project_root / "configs" / "chunking.yaml"
with open(config_path) as f:
    cfg = yaml.safe_load(f)

chroma_path = project_root / cfg["vector_store"].get("path", "./local_chroma")
print(f"ChromaDB path: {chroma_path}")

# Connect to ChromaDB
client = chromadb.PersistentClient(path=str(chroma_path))

# List all collections
collections = client.list_collections()
print(f"📁 Found {len(collections)} collections:")

total_chunks_chroma = 0
collection_details = []

for collection in collections:
    count = collection.count()
    total_chunks_chroma += count
    collection_details.append((collection.name, count))
    print(f"  📂 {collection.name}: {count} chunks")

print(f"\n🔍 Expected chunks: {EXPECTED_CHUNKS}")
print(f"🔍 Total chunks in ChromaDB: {total_chunks_chroma}")

if total_chunks_chroma == EXPECTED_CHUNKS:
    print("✅ ChromaDB chunk count matches expected!")
elif total_chunks_chroma > EXPECTED_CHUNKS:
    print(f"⚠️  ChromaDB has {total_chunks_chroma - EXPECTED_CHUNKS} extra chunks (stale data)")
else:
    print(f"❌ ChromaDB is missing {EXPECTED_CHUNKS - total_chunks_chroma} chunks")



=== CHECKING CHROMADB ===
ChromaDB path: /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/local_chroma


2025-08-01 10:36:47,119 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-08-01 10:36:47,119 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


📁 Found 1 collections:
  📂 mdc_training_data: 4165 chunks

🔍 Expected chunks: 4165
🔍 Total chunks in ChromaDB: 4165
✅ ChromaDB chunk count matches expected!


In [4]:
# Summary and next steps
print("\n=== SUMMARY ===")
print(f"Expected chunks from latest successful run: {EXPECTED_CHUNKS}")
print(f"DuckDB chunks: {actual_chunks_db}")
print(f"ChromaDB chunks: {total_chunks_chroma}")

# Check for inconsistencies
duckdb_extra = actual_chunks_db - EXPECTED_CHUNKS if actual_chunks_db > EXPECTED_CHUNKS else 0
chromadb_extra = total_chunks_chroma - EXPECTED_CHUNKS if total_chunks_chroma > EXPECTED_CHUNKS else 0

if duckdb_extra > 0 or chromadb_extra > 0:
    print("\n🔍 STALE DATA DETECTED:")
    if duckdb_extra > 0:
        print(f"  - DuckDB has {duckdb_extra} stale chunks")
    if chromadb_extra > 0:
        print(f"  - ChromaDB has {chromadb_extra} stale chunks")
    
    print("\n💡 This explains the retrieval failures:")
    print("  - ChromaDB returns stale chunk IDs during similarity search")
    print("  - DuckDB lookup fails for these stale IDs") 
    print("  - Result: 0 chunks retrieved → marked as unsuccessful")
    
    if chromadb_extra > 0:
        print(f"\n📊 ChromaDB Collection Details:")
        for name, count in collection_details:
            print(f"  📂 {name}: {count} chunks")
            
else:
    print("\n✅ No stale data detected!")
    print("💭 The 12 retrieval failures might be due to:")
    print("  - Citation pattern matching issues")  
    print("  - Citations not found in any chunks")
    print("  - Other query construction problems")



=== SUMMARY ===
Expected chunks from latest successful run: 4165
DuckDB chunks: 4205
ChromaDB chunks: 4165

🔍 STALE DATA DETECTED:
  - DuckDB has 40 stale chunks

💡 This explains the retrieval failures:
  - ChromaDB returns stale chunk IDs during similarity search
  - DuckDB lookup fails for these stale IDs
  - Result: 0 chunks retrieved → marked as unsuccessful


In [5]:
# ============================================================================
# PART 2: CITATION PATTERN MATCHING ANALYSIS
# ============================================================================

print("=== ANALYZING CITATION PATTERN MATCHING ===")

# Get all citations from the citations table
all_citations = db_helper.get_all_citation_entities()
print(f"📊 Total citations in database: {len(all_citations)}")

# Get all citations that are actually assigned to chunks
citations_in_chunks = set()
all_chunks = []

# Query all chunks to collect citation entities
result = db_helper.engine.execute("SELECT chunk_id, document_id, chunk_metadata FROM chunks")
chunk_rows = result.fetchall()
col_names = [desc[0] for desc in result.description]

print(f"📊 Processing {len(chunk_rows)} chunks to find assigned citations...")

for row in chunk_rows:
    row_dict = dict(zip(col_names, row))
    chunk_metadata = row_dict["chunk_metadata"]
    
    if chunk_metadata and "citation_entities" in chunk_metadata:
        citation_strings = chunk_metadata["citation_entities"]
        if citation_strings:  # Check if not empty list
            for ce_str in citation_strings:
                if ce_str:  # Check if not empty string
                    # Extract just the data_citation part (first part before |)
                    data_citation = ce_str.split("|")[0] if "|" in ce_str else ce_str
                    citations_in_chunks.add(data_citation)

print(f"📊 Citations found in chunks: {len(citations_in_chunks)}")
print(f"📊 Missing citations: {len(all_citations) - len(citations_in_chunks)}")

# Find the missing citations
all_citation_ids = {ce.data_citation for ce in all_citations}
missing_citations = all_citation_ids - citations_in_chunks

print(f"🔍 Expected missing: 49")
print(f"🔍 Actual missing: {len(missing_citations)}")

if len(missing_citations) > 0:
    print(f"\n📋 First 10 missing citations:")
    for i, citation in enumerate(list(missing_citations)[:10]):
        print(f"  {i+1}. {citation}")
else:
    print("✅ No missing citations found!")


2025-08-01 10:36:47,297 - src.helpers - INFO - Retrieved 487 citation entities from database
2025-08-01 10:36:47,297 - src.helpers - INFO - Retrieved 487 citation entities from database


=== ANALYZING CITATION PATTERN MATCHING ===
📊 Total citations in database: 487
📊 Processing 4205 chunks to find assigned citations...
📊 Citations found in chunks: 487
📊 Missing citations: 0
🔍 Expected missing: 49
🔍 Actual missing: 0
✅ No missing citations found!


In [6]:
# Test pattern matching on missing citations
from api.services.chunking_and_embedding_services import make_pattern
from src.helpers import preprocess_text
import re

print("\n=== TESTING PATTERN MATCHING ===")

if len(missing_citations) > 0:
    # Take first 3 missing citations for detailed analysis
    test_citations = list(missing_citations)[:3]
    
    for i, missing_citation in enumerate(test_citations, 1):
        print(f"\n🔍 TESTING CITATION {i}: {missing_citation}")
        
        # Find the document that contains this citation
        citation_obj = None
        for ce in all_citations:
            if ce.data_citation == missing_citation:
                citation_obj = ce
                break
        
        if not citation_obj:
            print("❌ Citation object not found")
            continue
            
        print(f"📄 Document: {citation_obj.document_id}")
        
        # Get the document's chunks
        doc_chunks = db_helper.get_chunks_by_document_id(citation_obj.document_id)
        print(f"📊 Document has {len(doc_chunks)} chunks")
        
        # Test the pattern matching
        pattern = make_pattern(missing_citation)
        print(f"🔍 Pattern: {pattern.pattern}")
        
        # Check if citation appears in any chunk text (raw and preprocessed)
        found_in_raw = 0
        found_in_preprocessed = 0
        found_with_pattern = 0
        
        for chunk in doc_chunks[:5]:  # Check first 5 chunks to avoid too much output
            raw_text = chunk.text
            preprocessed_text = preprocess_text(raw_text)
            
            # Check raw text
            if missing_citation.lower() in raw_text.lower():
                found_in_raw += 1
                print(f"  ✅ Found in raw text of chunk {chunk.chunk_id}")
                
            # Check preprocessed text  
            if missing_citation.lower() in preprocessed_text.lower():
                found_in_preprocessed += 1
                print(f"  ✅ Found in preprocessed text of chunk {chunk.chunk_id}")
                
            # Check pattern matching
            if pattern.search(preprocessed_text):
                found_with_pattern += 1
                print(f"  ✅ Pattern matched in chunk {chunk.chunk_id}")
        
        print(f"📊 Summary for {missing_citation}:")
        print(f"  Raw text matches: {found_in_raw}")
        print(f"  Preprocessed matches: {found_in_preprocessed}")
        print(f"  Pattern matches: {found_with_pattern}")
        
        if found_in_raw > 0 and found_with_pattern == 0:
            print("⚠️  ISSUE: Citation found in raw text but pattern didn't match!")
        elif found_in_raw == 0:
            print("⚠️  ISSUE: Citation not found in any chunk text - may be in different document!")

else:
    print("✅ No missing citations to test!")


/Users/taishajoseph/miniconda3/envs/mdc-challenge-2025/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-01 10:37:08,919 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/semantic_chunking.log
2025-08-01 10:37:08,919 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/semantic_chunking.log
2025-08-01 10:37:08,919 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/semantic_chunking.log
2025-08-01 10:37:08,925 - src.helpers - INFO - Logging initialized for /Users/taishajoseph/Documents/Projects/MDC-Challenge-2025/logs/semantic_chunking.log
2025-08-01 10:37:08,925 - src.helpers - INFO - Logging initialized for /Users/taishajos


=== TESTING PATTERN MATCHING ===
✅ No missing citations to test!


In [7]:
# Detailed analysis of text preprocessing effects
print("\n=== ANALYZING TEXT PREPROCESSING EFFECTS ===")

if len(missing_citations) > 0:
    # Take one missing citation for detailed preprocessing analysis
    test_citation = list(missing_citations)[0]
    print(f"🔍 DETAILED ANALYSIS FOR: {test_citation}")
    
    # Find the citation object
    citation_obj = None
    for ce in all_citations:
        if ce.data_citation == test_citation:
            citation_obj = ce
            break
    
    if citation_obj:
        print(f"📄 Document: {citation_obj.document_id}")
        
        # Get a chunk that might contain this citation
        doc_chunks = db_helper.get_chunks_by_document_id(citation_obj.document_id)
        
        # Find a chunk that contains the citation in raw text
        target_chunk = None
        for chunk in doc_chunks:
            if test_citation.lower() in chunk.text.lower():
                target_chunk = chunk
                break
        
        if target_chunk:
            print(f"📝 Found citation in chunk: {target_chunk.chunk_id}")
            
            # Show the text around the citation
            raw_text = target_chunk.text
            citation_pos = raw_text.lower().find(test_citation.lower())
            
            if citation_pos >= 0:
                # Extract text around the citation (±100 characters)
                start = max(0, citation_pos - 100)
                end = min(len(raw_text), citation_pos + len(test_citation) + 100)
                context = raw_text[start:end]
                
                print(f"\n📖 CONTEXT (raw text):")
                print(f"...{context}...")
                
                # Show preprocessed version
                preprocessed_context = preprocess_text(context)
                print(f"\n🔧 CONTEXT (preprocessed):")
                print(f"...{preprocessed_context}...")
                
                # Test different pattern approaches
                print(f"\n🧪 PATTERN TESTING:")
                
                # Original pattern
                original_pattern = make_pattern(test_citation)
                print(f"1. Original pattern: {original_pattern.pattern}")
                print(f"   Matches preprocessed: {bool(original_pattern.search(preprocessed_context))}")
                
                # Simple literal search
                print(f"2. Simple literal search in preprocessed:")
                print(f"   '{test_citation}' in preprocessed: {test_citation.lower() in preprocessed_context.lower()}")
                
                # Show what make_pattern does to the citation
                normalized_citation = re.sub(r'[^\w\s]', '', test_citation).lower()
                print(f"3. Pattern normalization:")
                print(f"   Original: '{test_citation}'")
                print(f"   Normalized: '{normalized_citation}'")
                print(f"   Normalized in preprocessed: {normalized_citation in preprocessed_context.lower()}")
                
        else:
            print("❌ Citation not found in any chunk of this document")
            
            # Check if citation exists in the full document text
            document = db_helper.get_documents_by_doi([citation_obj.document_id])
            if document:
                doc_text = " ".join(document[0].full_text) if isinstance(document[0].full_text, list) else document[0].full_text
                if test_citation.lower() in doc_text.lower():
                    print("⚠️  Citation exists in full document but not in any chunk!")
                    print("⚠️  This suggests chunking might have split the citation")
                else:
                    print("❌ Citation not found in full document either")

else:
    print("✅ No missing citations to analyze!")



=== ANALYZING TEXT PREPROCESSING EFFECTS ===
✅ No missing citations to analyze!


In [8]:
# Summary and recommendations
print("\n" + "="*80)
print("FINAL SUMMARY & RECOMMENDATIONS")
print("="*80)

print(f"\n📊 CITATION ANALYSIS RESULTS:")
print(f"  • Total citations in database: {len(all_citations)}")
print(f"  • Citations assigned to chunks: {len(citations_in_chunks)}")
print(f"  • Missing citations: {len(missing_citations)}")

print(f"\n📊 DATABASE CONSISTENCY:")
print(f"  • Expected chunks: {EXPECTED_CHUNKS}")
print(f"  • DuckDB chunks: {actual_chunks_db} (+{actual_chunks_db - EXPECTED_CHUNKS} stale)")
print(f"  • ChromaDB chunks: {total_chunks_chroma} ({total_chunks_chroma - EXPECTED_CHUNKS:+d})")

print(f"\n🔍 ROOT CAUSE ANALYSIS:")

if len(missing_citations) == (len(all_citations) - len(citations_in_chunks)):
    print("✅ Missing citation count matches expectation")
    print("🎯 CONFIRMED: Both issues stem from citation pattern matching failures")
    
    print(f"\n💡 ISSUE #1 (485→436): {len(missing_citations)} citations not assigned to chunks")
    print("💡 ISSUE #2 (12 failures): Some of these missing citations are queried in retrieval")
    print("   └─ get_query_texts() returns empty for missing citations")
    print("   └─ Leads to 0 chunks retrieved → marked as unsuccessful")
    
    print(f"\n🔧 RECOMMENDED FIXES:")
    print("1. Fix make_pattern() function:")
    print("   - Current: removes ALL punctuation, may be too aggressive")
    print("   - Consider: preserve important separators like / - .")
    print("   - Test: word boundaries may not work for complex identifiers")
    
    print("2. Improve text preprocessing:")
    print("   - Check if preprocess_text() is breaking citation formats")
    print("   - Consider matching against both raw AND preprocessed text")
    
    print("3. Add citation repair mechanism:")
    print("   - Check for citations split across chunk boundaries")
    print("   - Implement fuzzy matching for partial citations")
    
    print("4. Database cleanup:")
    print(f"   - Remove {actual_chunks_db - EXPECTED_CHUNKS} stale chunks from DuckDB")
    print("   - Ensure ChromaDB has complete chunk set")

else:
    print("⚠️  Citation count mismatch detected - need further investigation")

print(f"\n🚀 NEXT STEPS:")
print("1. Run this diagnostic on a few more missing citations")
print("2. Test improved pattern matching functions")
print("3. Clean up stale database entries")
print("4. Re-run chunking pipeline with fixes")

print("\n" + "="*80)



FINAL SUMMARY & RECOMMENDATIONS

📊 CITATION ANALYSIS RESULTS:
  • Total citations in database: 487
  • Citations assigned to chunks: 487
  • Missing citations: 0

📊 DATABASE CONSISTENCY:
  • Expected chunks: 4165
  • DuckDB chunks: 4205 (+40 stale)
  • ChromaDB chunks: 4165 (+0)

🔍 ROOT CAUSE ANALYSIS:
✅ Missing citation count matches expectation
🎯 CONFIRMED: Both issues stem from citation pattern matching failures

💡 ISSUE #1 (485→436): 0 citations not assigned to chunks
💡 ISSUE #2 (12 failures): Some of these missing citations are queried in retrieval
   └─ get_query_texts() returns empty for missing citations
   └─ Leads to 0 chunks retrieved → marked as unsuccessful

🔧 RECOMMENDED FIXES:
1. Fix make_pattern() function:
   - Current: removes ALL punctuation, may be too aggressive
   - Consider: preserve important separators like / - .
   - Test: word boundaries may not work for complex identifiers
2. Improve text preprocessing:
   - Check if preprocess_text() is breaking citation f

In [9]:
chunk = db_helper.get_chunks_by_chunk_ids(["10.1136_jitc-2021-003114_5"])
chunk[0].chunk_metadata

ChunkMetadata(chunk_id='10.1136_jitc-2021-003114_5', previous_chunk_id='10.1136_jitc-2021-003114_4', next_chunk_id='10.1136_jitc-2021-003114_6', token_count=608, citation_entities=[CitationEntity(data_citation='CVCL_0395', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_0530', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_0627', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_1916', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_1917', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_1918', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_1919', document_id='10.1136_jitc-2021-003114', pages=[2], evidence=None), CitationEntity(data_citation='CVCL_2213', document_id='1

In [10]:
# Verify separation of issues
import json

# Load retrieval results
retrieval_results_path = project_root / "reports" / "retrieval" / "retrieval_results.json"
with open(retrieval_results_path, 'r') as f:
    retrieval_data = json.load(f)

# Find failed retrievals
chunk_ids_map = retrieval_data.get('chunk_ids', {})
failed_retrievals = [cid for cid, chunks in chunk_ids_map.items() if not chunks or len(chunks) == 0]

print(f"📊 Verification Results:")
print(f"  • Citations attempted in retrieval: {len(chunk_ids_map)}")
print(f"  • Failed retrievals: {len(failed_retrievals)}")
print(f"  • Missing citations (never reached retrieval): {len(missing_citations)}")

# Check overlap
overlap = set(failed_retrievals) & missing_citations
print(f"  • Overlap between failed and missing: {len(overlap)}")

if len(overlap) == 0:
    print("✅ CONFIRMED: These are two separate issues")
    print(f"📊 Total problems: {len(missing_citations)} + {len(failed_retrievals)} = {len(missing_citations) + len(failed_retrievals)}")

📊 Verification Results:
  • Citations attempted in retrieval: 487
  • Failed retrievals: 0
  • Missing citations (never reached retrieval): 0
  • Overlap between failed and missing: 0
✅ CONFIRMED: These are two separate issues
📊 Total problems: 0 + 0 = 0


In [11]:
db_helper.close()

2025-08-01 10:37:09,079 - src.helpers - INFO - Database connection closed
2025-08-01 10:37:09,079 - src.helpers - INFO - Database connection closed
2025-08-01 10:37:09,079 - src.helpers - INFO - Database connection closed
2025-08-01 10:37:09,079 - src.helpers - INFO - Database connection closed
2025-08-01 10:37:09,079 - src.helpers - INFO - Database connection closed


In [12]:
len(chunk_ids_map)

487

In [14]:
# get list of n chunks per citation in chunk_ids_map
n_chunks_list = [len(chunks) for chunks in chunk_ids_map.values()]
print(f"Min chunks across all citations: {min(n_chunks_list)}")
print(f"Max chunks across all citations: {max(n_chunks_list)}")

Min chunks across all citations: 3
Max chunks across all citations: 6
